# Piping Chains and the RunnablePassthrough Class

In [13]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


In [14]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books.
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers.
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.

     Consider the following literature:
     {books}

     Also, consider the following projects:
     {projects}

     Roughly how much time would it take me to complete the literature and the projects?

     '''
)

In [15]:
chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [16]:
chat = ChatOpenAI(seed=365,
                  temperature = 0,
                  max_tokens = 100)

In [17]:
string_parser = StrOutputParser()

In [18]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [19]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [20]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a web scraper to extract data from websites and store it in a database.\n2. Creating a chatbot using natural language processing techniques to interact with users.\n3. Developing a machine learning model to predict stock prices based on historical data.'}

In [21]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [22]:
chain_time1 = (RunnableParallel({'books':chain_books,
                                'projects':chain_projects})
              | chat_template_time
              | chat
              | string_parser
             )

In [23]:
chain_time2 = ({'books':chain_books,
                'projects':chain_projects}
              | chat_template_time
              | chat
              | string_parser
             )

In [24]:
print(chain_time2.invoke({'programming language':'Python'}))

Completing the literature mentioned would depend on your reading speed and comprehension level. "Fluent Python" is around 792 pages, "Python Cookbook" is around 706 pages, and "Effective Python" is around 256 pages. If you dedicate a few hours each day to reading, it could take you a few weeks to a couple of months to finish all three books.

As for the projects, the time required would vary based on your familiarity with the technologies involved and the complexity of the tasks


In [25]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            